In [1]:
%run ../__init__.py

## Data preprocessing with Tfidf

In [2]:
budget_train = pd.read_csv('../data/train_cleaned.csv', index_col=0)
budget_test = pd.read_csv('../data/test_clean.csv', index_col=0)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
budget_train.isnull().sum()

Function                      0
Use                           0
Sharing                       0
Reporting                     0
Student_Type                  0
Position_Type                 0
Object_Type                   0
Pre_K                         0
Operating_Status              0
Object_Description          423
Text_2                       17
SubFund_Description           0
Job_Title_Description         0
Text_3                    70814
Text_4                       18
Sub_Object_Description        0
Location_Description          0
FTE                           0
Function_Description          5
Facility_or_Department        0
Position_Extra              611
Total                         0
Program_Description           2
Fund_Description              0
Text_1                      422
combined_text                 0
dtype: int64

In [4]:
budget_test.isnull().sum()

ID                            0
Object_Description         1752
Program_Description        5253
SubFund_Description       33953
Job_Title_Description     17747
Facility_or_Department    47225
Sub_Object_Description    16452
Location_Description      12748
FTE                           0
Function_Description       3198
Position_Extra            36279
Text_4                    47252
Total                         0
Text_2                    45426
Text_3                    44281
Fund_Description          10478
Text_1                    34717
dtype: int64

In [5]:
for col in budget_test[word_predictors]:
    budget_test[col] = budget_test[word_predictors].loc[:,[col]].fillna(' ')

In [6]:
for col in budget_train[word_predictors]:
    budget_train[col] = budget_train[word_predictors].loc[:,[col]].fillna(' ')

In [7]:
label = LabelEncoder()

In [8]:
class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    Parameters
     ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]
    

In [9]:
class ArrayCaster(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, data):
        return np.transpose(np.matrix(data))

In [10]:
targets

['Function',
 'Use',
 'Sharing',
 'Reporting',
 'Student_Type',
 'Position_Type',
 'Object_Type',
 'Pre_K',
 'Operating_Status']

In [11]:
# 400,000+ examples, 14 features
X = budget_train[predictors] 
y = label.fit_transform(budget_train['Student_Type'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = .2, random_state=42)
                                                

In [12]:
# feature union transformer list 
transformer_list = [
    ('Object_Description', Pipeline([
                ('selector', ItemSelector(key='Object_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_2', Pipeline([
                ('selector', ItemSelector(key='Text_2')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('SubFund_Description', Pipeline([
                ('selector', ItemSelector(key='SubFund_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Job_Title_Description', Pipeline([
                ('selector', ItemSelector(key='Job_Title_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_3', Pipeline([
                ('selector', ItemSelector(key='Text_3')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_4', Pipeline([
                ('selector', ItemSelector(key='Text_4')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Sub_Object_Description', Pipeline([
                ('selector', ItemSelector(key='Sub_Object_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Location_Description', Pipeline([
                ('selector', ItemSelector(key='Location_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Function_Description', Pipeline([
                ('selector', ItemSelector(key='Function_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Facility_or_Department', Pipeline([
                ('selector', ItemSelector(key='Facility_or_Department')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Position_Extra', Pipeline([
                ('selector', ItemSelector(key='Position_Extra')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Program_Description', Pipeline([
                ('selector', ItemSelector(key='Program_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Fund_Description', Pipeline([
                ('selector', ItemSelector(key='Fund_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
    ('Text_1', Pipeline([
                ('selector', ItemSelector(key='Text_1')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=100)),
            ])),
#     ('FTE', Pipeline([
#                 ('selector', ItemSelector(key='FTE')),
#                 ('scaler', StandardScaler()),
#                 ('caster', ArrayCaster()),
#             ])),
#     ('Total', Pipeline([
#                 ('selector', ItemSelector(key='Total')),
#                 ('scaler', StandardScaler()),
#                 ('caster', ArrayCaster()),
#             ])),
]

### Don't include numerical columns in final feature union since it decreased scores and increased log loss- in general, in the original data, numerical features had a lot of missing values and some incorrect values for data that were there

In [13]:
# create pipeline
model_pipeline = Pipeline([
    ("union", FeatureUnion(transformer_list=transformer_list)),
    ("clf", LogisticRegression()),
])

In [14]:
X_test = budget_test

In [15]:
student_model = model_pipeline.fit(X_train, y_train)

In [16]:
student_model.score(X_train, y_train)

0.97795584924161749

In [17]:
val_proba = student_model.predict_proba(X_val)

In [18]:
predict_proba = student_model.predict_proba(X_test)

In [19]:
label.classes_

array(['Student_Type__Alternative', 'Student_Type__At Risk',
       'Student_Type__ELL', 'Student_Type__Gifted',
       'Student_Type__NO_LABEL', 'Student_Type__Poverty',
       'Student_Type__PreK', 'Student_Type__Special Education',
       'Student_Type__Unspecified'], dtype=object)

In [20]:
log_loss(y_val, val_proba)

0.079314984649002634

In [21]:
from sklearn.externals import joblib

In [22]:
joblib.dump(student_model, 'student_model.pkl')

['student_model.pkl']

In [23]:
def score_eval(model, X_val, y_val):
    preds = model.predict(X_val)
    proba = model.predict_proba(X_val)
    return {
        'log_loss': log_loss(y_val, proba),
        'accuracy': accuracy_score(y_val, preds),
        'confmat': confusion_matrix(y_val, preds),
        'clf_rep': classification_report(y_val, preds)
    }

In [24]:
results = score_eval(student_model, X_val, y_val)

print('-'*60)
for key, value in results.items():
    print(key)
    print(value)
    print('-'*60)

------------------------------------------------------------
log_loss
0.079314984649
------------------------------------------------------------
accuracy
0.975404716698
------------------------------------------------------------
confmat
[[   67     0     0     0     9     1     0     2    23]
 [    0   688     0     0     0     0     0     0     2]
 [    0     0  1179     0     2    19     2     0   169]
 [    0     0     0   312     0     0     0     0    10]
 [    8     8     9     2 19476    31     2    45   454]
 [    0     8     0     0    12  3467     0     0     5]
 [    0     0    16     0     0     0  1055     0    11]
 [    1     1     4     2    37    14     1  8146   395]
 [    0     0    23     0   313     6     3   319 43697]]
------------------------------------------------------------
clf_rep
             precision    recall  f1-score   support

          0       0.88      0.66      0.75       102
          1       0.98      1.00      0.99       690
          2       

In [25]:
classes = label.classes_

In [28]:
student_probability = pd.DataFrame(predict_proba, columns=classes)

In [29]:
student_probability.head()

,Student_Type__Alternative,Student_Type__At Risk,Student_Type__ELL,Student_Type__Gifted,Student_Type__NO_LABEL,Student_Type__Poverty,Student_Type__PreK,Student_Type__Special Education,Student_Type__Unspecified
0,0.000187,0.000020,0.000091,0.000079,0.073396,0.000073,0.000007,0.001721,0.924427
1,0.000105,0.000027,0.000053,0.000731,0.032570,0.000014,0.000009,0.000734,0.965758
2,0.000145,0.000271,0.003752,0.000326,0.059441,0.000072,0.000007,0.002389,0.933597
3,0.000145,0.000271,0.003736,0.000325,0.059079,0.000073,0.000007,0.002398,0.933965
4,0.000120,0.000049,0.000006,0.000128,0.005819,0.000020,0.000005,0.000846,0.993008


In [30]:
student_probability.to_csv('student_type_probability.csv')